In [1]:
from konlpy.tag import Kkma, Komoran
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [71]:
df = pd.read_csv('corpus_df.csv', encoding = 'cp949')
df['fulltext'] = df['fulltext'].str.strip()
df['fulltext'] = df['fulltext'].str.replace('\n',"")

In [72]:
df.head()

date                                           fulltext  \
0  2019-06-13 16:29:00              500_02001999574350488884640사전 예약 중입니다   
1  2019-06-07 14:40:00                                              56월쿠폰   
2  2019-05-31 20:07:00  이쁜거만 올릴까 하다가 좀 그래서 재밌는거도 가져왔음 ㅋㅋ 첫번째는 왕좌의 게임 존...   
3  2019-05-29 14:43:00                    44 에서 쿠폰번호 등록여태 잊어먹었네요 잘쓰석요5월쿠폰   
4  2019-05-29 14:00:00                                       하고싶어요직업추천받아요   

  source  
0  INVEN  
1  INVEN  
2  INVEN  
3  INVEN  
4  INVEN

In [73]:
df['fulltext'] = df['fulltext'].str.strip()

In [74]:
df = df.loc[df['fulltext'].str.len() > 1, :]
doc_to_mat = df['fulltext'].values
doc_to_mat = list(doc_to_mat)

In [84]:
komoran = Komoran()
komoran.tagset

{'EC': '연결 어미',
 'EF': '종결 어미',
 'EP': '선어말어미',
 'ETM': '관형형 전성 어미',
 'ETN': '명사형 전성 어미',
 'IC': '감탄사',
 'JC': '접속 조사',
 'JKB': '부사격 조사',
 'JKC': '보격 조사',
 'JKG': '관형격 조사',
 'JKO': '목적격 조사',
 'JKQ': '인용격 조사',
 'JKS': '주격 조사',
 'JKV': '호격 조사',
 'JX': '보조사',
 'MAG': '일반 부사',
 'MAJ': '접속 부사',
 'MM': '관형사',
 'NA': '분석불능범주',
 'NF': '명사추정범주',
 'NNB': '의존 명사',
 'NNG': '일반 명사',
 'NNP': '고유 명사',
 'NP': '대명사',
 'NR': '수사',
 'NV': '용언추정범주',
 'SE': '줄임표',
 'SF': '마침표, 물음표, 느낌표',
 'SH': '한자',
 'SL': '외국어',
 'SN': '숫자',
 'SO': '붙임표(물결,숨김,빠짐)',
 'SP': '쉼표,가운뎃점,콜론,빗금',
 'SS': '따옴표,괄호표,줄표',
 'SW': '기타기호 (논리수학기호,화폐기호)',
 'VA': '형용사',
 'VCN': '부정 지정사',
 'VCP': '긍정 지정사',
 'VV': '동사',
 'VX': '보조 용언',
 'XPN': '체언 접두사',
 'XR': '어근',
 'XSA': '형용사 파생 접미사',
 'XSN': '명사파생 접미사',
 'XSV': '동사 파생 접미사'}

In [77]:
stopwords = [
    "이다", "있다", '진짜', '입니다', '게임','하나','모바일',
    '라그나로크', '라그나로크 사랑', '라그나로크 모바일', '모바일 라그나로크','라그', '사랑',
    '오늘', '한번','요즘','때문','정도', '같다','어떻다', '아니다', "그렇다", "이렇다"
    ]
def komoran_tokenizer(raw, stopword=stopwords): #품사 지정 
    return [
        word for word, tag in komoran.pos(raw,flatten=True)
            if len(word) > 1 and tag in ["NNG","NNP"] and word not in stopword
        ]

In [80]:
tfidfvectorizer = TfidfVectorizer(tokenizer=komoran_tokenizer, max_df=df.shape[0]/2,min_df=10, ngram_range=(1,2))    # tf값에 1+log(tf)를 적용하여 tf값이 무한정 커지는 것을 막음)
tfidfvector = tfidfvectorizer.fit_transform(doc_to_mat)
print('fit_transform, (sentence {}, feature {})'.format(tfidfvector.shape[0], tfidfvector.shape[1]))
print(tfidfvector.toarray())
features = tfidfvectorizer.get_feature_names()
#print(features)

fit_transform, (sentence 14630, feature 1701)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [81]:
tfidf_vec_df2 = pd.DataFrame(tfidfvector.toarray(), columns=features)
tfidf_vec_df2.head()

10  10분  15년   1년  20대   2시   30   3월  3월 14일   40  ...   후기   후반   후에  \
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  ...  0.0  0.0  0.0   
1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  ...  0.0  0.0  0.0   
2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  ...  0.0  0.0  0.0   
3  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  ...  0.0  0.0  0.0   
4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  ...  0.0  0.0  0.0   

    후회   훈장  휴대폰   흥미   흥행   힐러   힐링  
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 1701 columns]

In [83]:
max_tfidf = tfidf_vec_df2.iloc[:,:-2].sum(axis=0).sort_values()
print("tf-idf 상위 50 token: \n{}".format(max_tfidf[-50:]))

tf-idf 상위 50 token: 
계정        76.815505
레벨        77.538537
설치        77.565648
광고        77.741879
다운로드      79.403060
느낌        79.425372
화면        81.204016
니다        82.918012
길드        83.085209
그라비티      83.304312
감사        83.561427
서강준       85.224272
오픈        85.278768
스키        86.814013
아이템       87.695149
다운        88.630420
패치        89.915318
확인        90.200525
캐릭터       95.524954
실행        96.217651
카드        96.498614
쿠폰       101.054917
플레이      102.896212
예전       103.528423
영원       107.044773
부탁       109.205533
처음       110.899870
운영       111.771239
자동       113.466571
사람       114.964432
최고       116.071814
이벤트      118.956622
향수       122.374754
사냥       128.334215
접속       135.992713
시작       136.492593
윤아       139.712774
가요       142.099559
사전 예약    144.966389
예약       148.635902
과금       152.406553
사전       160.304649
옛날       163.692437
기대       165.826540
퀘스트      171.532124
시간       177.335673
생각       193.957458
업데이트     204.531478
재미       409.093117

In [85]:
def komoran_tokenizer2(raw, stopword=stopwords): #고유명사 NNP
    return [
        word for word, tag in komoran.pos(raw,flatten=True)
            if len(word) > 1 and tag in ["NNP"] and word not in stopword
        ]

In [89]:
tfidfvectorizer = TfidfVectorizer(tokenizer=komoran_tokenizer2, max_df=df.shape[0]/2,min_df=10, ngram_range=(1,1))    # tf값에 1+log(tf)를 적용하여 tf값이 무한정 커지는 것을 막음)
tfidfvector = tfidfvectorizer.fit_transform(doc_to_mat)
print('fit_transform, (sentence {}, feature {})'.format(tfidfvector.shape[0], tfidfvector.shape[1]))
print(tfidfvector.toarray())
features = tfidfvectorizer.get_feature_names()
#print(features)
tfidf_vec_df = pd.DataFrame(tfidfvector.toarray(), columns=features)
tfidf_vec_df.head()

fit_transform, (sentence 14630, feature 711)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


10  10분  15년   1년  20대   2시   30   3월  3월 14일   40  ...   화살  화이트데이  화이팅  \
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  ...  0.0    0.0  0.0   
1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  ...  0.0    0.0  0.0   
2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  ...  0.0    0.0  0.0   
3  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  ...  0.0    0.0  0.0   
4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  ...  0.0    0.0  0.0   

    확률   회복   회사   후에   훈장   힐러   힐링  
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 711 columns]

In [90]:
max_tfidf = tfidf_vec_df.iloc[:,:-2].sum(axis=0).sort_values()
print("tf-idf 상위 50 token: \n{}".format(max_tfidf[-50:]))

tf-idf 상위 50 token: 
보스       55.952571
하네       55.958158
재미나      56.367290
몬스터      57.421295
강타       58.813104
고양이      59.260139
사전       60.258013
파티       60.292972
뽑기       60.693267
오류       64.332515
순위       64.757922
중국       65.924421
이랑       68.109181
피로       69.720731
의상       70.387091
스키       72.657262
이건       74.900942
해주       82.353785
버전       87.792610
그래픽      88.070718
서버       92.308284
시간       92.632717
제가       94.391019
다운로드     97.004596
접속       97.350271
향수       97.856064
계정      101.348082
예전      105.369878
오픈      106.746068
플레이     107.785177
다운      108.038715
로딩      108.974931
레벨      109.880329
패치      114.941171
서강준     116.388461
길드      121.315469
그라비티    126.345893
니다      128.357412
사냥      131.370984
캐릭터     136.359730
아이템     136.691239
이벤트     164.581222
쿠폰      170.271286
윤아      187.749575
가요      204.790132
과금      219.606354
퀘스트     263.207826
재미      266.942393
추억      292.529456
업데이트    294.786855
dtype: float64


In [88]:
kkma = Kkma()

In [91]:
def kkma_tokenizer(raw, stopword=stopwords): #품사 지정 
    return [
        word for word, tag in kkma.pos(raw,flatten=True)
            if len(word) > 1 and tag in ["NNG","NNP"] and word not in stopword
        ]

In [ ]:
tfidfvectorizer = TfidfVectorizer(tokenizer=kkma_tokenizer, max_df=df.shape[0]/2,min_df=10, ngram_range=(1,2))    # tf값에 1+log(tf)를 적용하여 tf값이 무한정 커지는 것을 막음)
tfidfvector = tfidfvectorizer.fit_transform(doc_to_mat)
print('fit_transform, (sentence {}, feature {})'.format(tfidfvector.shape[0], tfidfvector.shape[1]))
print(tfidfvector.toarray())
features = tfidfvectorizer.get_feature_names()
#print(features)
tfidf_vec_df = pd.DataFrame(tfidfvector.toarray(), columns=features)
tfidf_vec_df.head()

In [ ]:
max_tfidf = tfidf_vec_df.iloc[:,:-2].sum(axis=0).sort_values()
print("tf-idf 상위 50 token: \n{}".format(max_tfidf[-50:]))